## Módulo: Analytics Engineering
    
## Aula 3 - Parte 1

### Programação da Aula 3:

> ### 1. **O que é a "Great Expectations"**:
> ### 2. **"Great Expectations" na prática**;
> ### 3. **Apresentação do projeto**;
> ### 4. **Desenvolvimento de exercício ou projeto**;

#### Link para o formulário para informar os integrantes do grupo do projeto:
https://forms.gle/xgT27QH81dxSrZ1w7

#### Link para o formulário de Feedback da aula:
https://forms.gle/rpqeKYPxbG86g8Zy9

### Great Expectations

#### O Great Expectations (GE) é uma ferramenta de validação de dados open source que ajuda a garantir a qualidade dos dados.

#### Com o Great Expectations, é possível definir expectativas sobre seus dados e verificar se elas atendem ou não.

#### Algumas das funcionalidades existentes incluem a capacidade de validar o esquema dos dados, garantir a integridade referencial, verificar a consistência e detectar anomalias.

#### O GE é muito flexível e escalável, permitindo assim a integração nas nossas pipelines de dados, seja para validar, gerar relatórios ou até mesmo impedir que a pipeline avance gravando dados inconsistentes nas camadas mais “curadas” do Data Lake.

#### Alguns pontos que podemos destacar:

+ #### é possível criar testes de dados diretamente de dataframes criados com Pandas ou Spark;
+ #### é possível gerar uma documentação dos dados em HTML de forma automática, que traz diversas informações sobre os testes executados;
+ #### é possível salvar um conjunto de testes (suite) para serem utilizados posteriormente (checkpoints);
+ #### podemos utilizar uma grande quantidade de expectativas já prontas, ou criar facilmente expectativas personalizadas que atendam os nossos casos de testes;
+ #### possui uma CLI que facilita a criação dos casos de testes, ou simplemente, podemos criar codificando em Python;
+ #### é possível conectar diretamente a fontes de origem de dados, assim validar os dados de forma mais rápida.

<div style="text-align: center;">

<br>

<img src="https://docs.greatexpectations.io/assets/images/gx_oss_process-050a4264f415a1bff3ceea3ac6f9b3a0.png"  width="80%" height="60%">

###### Fonte: https://docs.greatexpectations.io/assets/images/gx_oss_process-050a4264f415a1bff3ceea3ac6f9b3a0.png

<br>

<img src="https://docs.greatexpectations.io/assets/images/datasource_flowchart-73d07084d7c20d480f92dacb8001222f.png"  width="80%" height="60%">

###### Fonte: https://docs.greatexpectations.io/assets/images/datasource_flowchart-73d07084d7c20d480f92dacb8001222f.png
<br>

<img src="https://docs.greatexpectations.io/assets/images/checkpoint_flowchart-80a2dc06e90a4739f6cfdec60a115c1e.png"  width="80%" height="60%">

###### Fonte: https://docs.greatexpectations.io/assets/images/checkpoint_flowchart-80a2dc06e90a4739f6cfdec60a115c1e.png
<br>

### Instalação da biblioteca "Great Expectations"

In [ ]:
pip install great_expectations

In [ ]:
pip show great_expectations

### Leitura de um arquivo csv com o Pandas direto pelo "great_expectations"

In [ ]:
import great_expectations as gx

df = gx.read_csv("F:\ADA\AnalyticsEngineer\lucio\dados\\autos.csv", encoding='ISO-8859-1')
df.head() #as funções funcionam igual

### Ou leitura do arquivo com o Pandas e depois convertando para o formato do "great_expectations"

In [ ]:
import pandas as pd

df = pd.read_csv("dados/autos.csv", encoding='ISO-8859-1')
df = gx.from_pandas(df) #de um dataframe pandas para o formato do "great_expectations"
df.head()

### Adiciona duas expectativas de tipo de coluna para a base de dados 

In [ ]:
#expectativa de que a coluna 'yearOfRegistration' precisa ser do tipo 'int64'
df.expect_column_values_to_be_of_type(column='yearOfRegistration', type_='int64')
#expectativa de que a coluna 'monthOfRegistration' precisa ser do tipo 'int64'
df.expect_column_values_to_be_of_type(column='monthOfRegistration', type_='int64')

### Lista com todas as expectativas disponíveis:
https://greatexpectations.io/expectations/

### As expectativas ficam armazenadas em um "Expectation Suite":

In [ ]:
df.get_expectation_suite()

### Adicionando uma nova expectativa, agora com relação a valores esperados de 1 a 12 para a coluna 'monthOfRegistration'

In [ ]:
df.expect_column_values_to_be_between(
    column="monthOfRegistration",
    min_value=1,  #valor minimo esperado
    max_value=12, #valor maximo esperado
)

### Agora trabalhando com checkpoints e validadores 

In [ ]:
import great_expectations as gx

context = gx.get_context() #inicia um contexto do "great_expectations"

print(context)

### Perceba que o contexto inicia sem nenhuma lista de Expectativas (as "Expectation Suites")

In [ ]:
context.list_expectation_suite_names()

### Cria um validador a partir da base de dados pelo arquivo csv e o Pandas

In [ ]:
validator = context.sources.pandas_default.read_csv("F:\ADA\AnalyticsEngineer\lucio\dados\\autos.csv", encoding='ISO-8859-1') #cria o validador

#expectativa de que a coluna 'yearOfRegistration' precisa ser do tipo 'int64'
validator.expect_column_values_to_be_of_type(column='yearOfRegistration', type_='int64') 
#expectativa de que a coluna 'monthOfRegistration' precisa ser do tipo 'int64'
validator.expect_column_values_to_be_of_type(column='monthOfRegistration', type_='int64')

#expectativa de valores esperados entre 1000 e 500000 para a coluna 'kilometer'
validator.expect_column_values_to_be_between(
    column="kilometer",
    min_value=1000,
    max_value=5000,
)

#expectativa de valores esperados entre 1900 e 2016 para a coluna 'yearOfRegistration'
validator.expect_column_values_to_be_between(
    column="yearOfRegistration",
    min_value=1900,
    max_value=2016,
)

### Salva a "Expectation Suite" criada pelo validador

In [41]:
#o parametro "discard_failed_expectations" igual a "False" indica que mesmos as expectativas que não foram cumpridas serão salvas
validator.save_expectation_suite(discard_failed_expectations=False) 

In [ ]:
#agora pode se observer que a "Expectation Suite" está disponível no "contexto"
context.list_expectation_suite_names()

### Cria um "checkpoint" a partir do resultado atual do validador

In [43]:
checkpoint = context.add_or_update_checkpoint(name="primeiro_checkpoint", validator=validator)

### Processa esse "checkpoint"

In [ ]:
checkpoint_result = checkpoint.run()

### Disponibiliza uma documentação web com as informações das expectativas e testes dos "checkpoints" no link indicado

In [ ]:
context.build_data_docs()

### Continuação dos testes com a adição de uma nova expectativa

In [ ]:
#expectativa de valores esperados entre 0 e 30000 para a coluna 'powerPS'
validator.expect_column_values_to_be_between(
    column="powerPS",
    min_value=0,
    max_value=300,
)

### Salva a novo conjunto de expectativas ("Expectation Suite") e processa mais um "checkpoint"

In [48]:
validator.save_expectation_suite(discard_failed_expectations=False)

In [50]:
checkpoint = context.add_or_update_checkpoint(name="segundo_checkpoint", validator=validator)

In [ ]:
checkpoint_result = checkpoint.run()